In [15]:
import torch
import torch.optim as optim

import torchvision
import matplotlib
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split

In [16]:
df = pd.read_csv('/content/winequality-red.csv', sep=';')
df.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5


In [17]:
#input featues. We will predict quality
X = torch.tensor(df.drop(columns=["quality"]).values).float()
X.shape

torch.Size([1599, 11])

In [18]:
#real output. 
y = torch.tensor(df["quality"].values).float()
y.shape

torch.Size([1599])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

In [20]:
# Bring the data in batch format
batchsize = 32
X_train = X_train[:-(X_train.shape[0] % batchsize)]
y_train = y_train[:-(y_train.shape[0] % batchsize)]
X_train = X_train.reshape(int(X_train.shape[0] / batchsize), batchsize, -1)
y_train = y_train.reshape(int(y_train.shape[0] / batchsize), batchsize, -1)

In [28]:
class Regression(nn.Module):
    def __init__(self, num_features):
        super(Regression, self).__init__()

        self.out = nn.Sequential(
            nn.Linear(num_features, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 1),
        )
        
        
    def forward(self, x):
        x = self.out(x)
        return x

model = Regression(11)
print(model)

Regression(
  (out): Sequential(
    (0): Linear(in_features=11, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)


In [29]:
lr = 0.001
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [34]:
from tqdm import trange
from torch.autograd import Variable
epochs = 1000
for epoch in trange(epochs):
    for X_batch, y_batch in zip(X_train, y_train):
       
        inputs = Variable((X_batch))
        labels = Variable((y_batch))

        optimizer.zero_grad()


        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    if epoch%200==0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))
print('epoch {}, loss {}'.format(epoch, loss.item()))

  1%|          | 8/1000 [00:00<00:27, 36.39it/s]

epoch 0, loss 0.3752274513244629


 21%|██        | 208/1000 [00:05<00:20, 38.21it/s]

epoch 200, loss 0.3722781836986542


 41%|████      | 408/1000 [00:10<00:15, 37.51it/s]

...

epoch 600, loss 0.35387879610061646


 81%|████████  | 807/1000 [00:21<00:05, 38.37it/s]

epoch 800, loss 0.3252658545970917


100%|██████████| 1000/1000 [00:26<00:00, 37.81it/s]

epoch 999, loss 0.29244375228881836


In [35]:
with torch.no_grad():
    predicted = model(Variable((X_test))).data.numpy()

In [36]:
predicted[0:5]

array([[6.650576 ],
       [5.100961 ],
       [6.625055 ],
       [5.6102805],
       [6.4271317]], dtype=float32)

In [37]:
y_test[0:5]

tensor([7., 5., 6., 5., 6.])